In [36]:
import pandas as pd
import datetime
import numpy as np
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
import pickle

In [37]:
directory = 'vers23/MLP_Classifiers_480k_training_200_iter_NN_size_200_200_50_50_30'
#directory = 'vers33/MLP_Classifiers_300k_training_10_iter_NN_size_200_200_50_50_30'
#use_z_values = tuple(range(25))
#use_z_values = (0,6,12,18,24)
use_z_values = (0,3,4,6,8,9,12,13,14,15,16,17,18,19,20,21,22,23,24)

# Indices in terms of z for all hidden variables that are not mixed, e.g. Subject1-Subject2, Verb1-Verb2, etc.
non_mixed_pairs_indices = [i for i in range(len(use_z_values)) if use_z_values[i] in (0,6,12,18,24)]

In [38]:
# Indices for all z variables influenced by Subject2 (Verb2, Object2 etc. respectively) (e.g. Subject1-Subject2)
Subj2_indices = [i for i in range(len(use_z_values)) if use_z_values[i] in (0, 5, 10, 15, 20)]
Verb2_indices = [i for i in range(len(use_z_values)) if use_z_values[i] in (1, 6, 11, 16, 21)]
Obj2_indices = [i for i in range(len(use_z_values)) if use_z_values[i] in (2, 7, 12, 17, 22)]
Loc2_indices = [i for i in range(len(use_z_values)) if use_z_values[i] in (3, 8, 13, 18, 23)]
Clo2_indices = [i for i in range(len(use_z_values)) if use_z_values[i] in (4, 9, 14, 19, 24)]
Sentence2_indices = [Subj2_indices, Verb2_indices, Obj2_indices, Loc2_indices, Clo2_indices]

In [39]:
train1 = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_train_1.csv')
train2 = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_train_2.csv')
train = pd.concat([train1, train2])
train = train[train.notnull().apply(all, axis=1)]
dev = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_dev.csv')
dev = dev[dev.notnull().apply(all, axis=1)]
test = pd.read_csv('../Input_Data/e-SNLI/dataset/esnli_test.csv')
test = test[test.notnull().apply(all, axis=1)]

dev_prepared = pd.read_csv('../02_Extract_Subphrases/prepared_data/subphrase_vectors_dev.csv', sep=';')
dev_prepared = dev_prepared.drop(columns='Unnamed: 0')
dev = dev.set_index('pairID')
rel_pairIDs = dev_prepared.iloc[:,0]
y_hat = dev.loc[rel_pairIDs].gold_label
dev_prepared = dev_prepared.iloc[:,1:].to_numpy()

In [40]:
dev_prepared.shape[0] / dev.shape[0]

0.3083722820564926

In [41]:
dev_prepared.shape[0]

3035

In [42]:
dev_subphrases = pd.read_csv('../02_Extract_Subphrases/prepared_data/subphrases_dev.csv', sep=',')
dev_subphrases = dev_subphrases.set_index('pairID')
dev_subphrases = dev_subphrases.loc[rel_pairIDs]

In [43]:
def evaluate_performance(preds, actual):
    labels = ['contradiction', 'entailment', 'neutral']
    print('Confusion Matrix')
    print(confusion_matrix(actual, preds, labels=labels))
    print(f'F1-Score: {f1_score(actual, preds, labels=labels, average="macro")}')
    print(f'Accuracy: {accuracy_score(actual, preds)}')

In [44]:
def predict_y_from_z(z):
    if len(z.shape) > 1:
        z = pd.DataFrame(z)
        res = z.apply(predict_y_from_z, axis=1)
        return res.to_numpy()
    else:
        if all([z[i] == 'nan' or pd.isnull(z[i]) or z[i] == 'entailment' for i in range(25)]):
            return 'entailment'
        elif any(z == 'contradiction'):
            return 'contradiction'
        elif any(z == 'neutral') and all(z != 'contradiction'):
            return 'neutral'
        else:
            raise ValueError(f"z can only have values 'entailment', 'contradiction', or 'neutral' but is {z}")

In [45]:
def predict_y_from_z(z):
    if len(z.shape) > 1:
        z = pd.DataFrame(z)
        res = z.apply(predict_y_from_z, axis=1)
        return res.to_numpy()
    else:
        if any(z == 'contradiction'):
            return 'contradiction'
        elif all([z[i] == 'nan' or pd.isnull(z[i]) or z[i] == 'entailment' for i in (0,6,12,18,24)]) and all(z != 'contradiction'):
            return 'entailment'
        elif any(z == 'neutral') and all(z != 'contradiction'):
            return 'neutral'
        else:
            raise ValueError(f"z can only have values 'entailment', 'contradiction', or 'neutral' but is {z}")

In [46]:
def predict_y_from_z(z):

    # Iterate through each row if z is a matrix
    if len(z.shape) > 1:
        z = pd.DataFrame(z)
        res = z.apply(predict_y_from_z, axis=1)
        return res.to_numpy()

    # For each single line perform the following:
    else:
        # If any z is 'contradiction' -> output class 'contradiction'
        if any(z == 'contradiction'):
            return 'contradiction'
        # Else if all subphrases of sentence 2 are entailed by any subphrase of sentence 1 -> output class 'entailment'
        elif all([any([z[i] == 'entailment' for i in subphrase_indices]) or all([z[i] == 'nan' or pd.isnull(z[i]) for i in subphrase_indices]) for subphrase_indices in Sentence2_indices]):
            return 'entailment'
        # Else output class 'neutral'
        else:
            return 'neutral'

In [47]:
clf = list()
for i in range(len(use_z_values)):
    with open("../03_Bayesian_Network/" + directory + "/MLP_Classifier" + str(i) + ".pkl", "rb") as f:
        clf += [pickle.load(f), ]

In [48]:
# Prepare colum indices
indices = np.array([[0,1500], [0,1800], [0,2100], [0,2400], [0,2700],
           [300,1500], [300,1800], [300,2100], [300,2400], [300,2700],
           [600,1500], [600,1800], [600,2100], [600,2400], [600,2700],
           [900,1500], [900,1800], [900,2100], [900,2400], [900,2700],
           [1200,1500], [1200,1800], [1200,2100], [1200,2400], [1200,2700]])
indices = indices[use_z_values,:].tolist()
# Initialise colulmn indices and "nan" values if information (e.g. location of sentence) is not detected
print(datetime.datetime.now())
print("Initialise column indices and 'nan' values")
not_nan = [None, ] * len(use_z_values)
cols = [None, ] * len(use_z_values)
for i in range(len(use_z_values)):
    cols[i] = list(range(indices[i][0], indices[i][0]+300)) + list(range(indices[i][1],indices[i][1]+300))
    not_nan[i] = pd.Series([not x for x in pd.DataFrame(np.isnan(dev_prepared[:,cols[i]])).apply(any, axis=1)])
not_nan = np.array(not_nan).T

2023-03-19 12:45:21.567064
Initialise column indices and 'nan' values


In [49]:
z = np.empty((y_hat.shape[0], len(use_z_values)), dtype=np.dtype('U25'))

z[:,:] = np.nan

for i in range(len(use_z_values)):
    z[not_nan[:,i], i] = clf[i].predict(dev_prepared[not_nan[:,i],:][:, cols[i]])

y_hat_pred = predict_y_from_z(z)

In [50]:
def generate_explanation(Z, subphrases):
    y_hat_pred = predict_y_from_z(Z)
    pairs = [['subj_s1', 'subj_s2'], ['subj_s1', 'verb_s2'], ['subj_s1', 'obj_s2'], ['subj_s1', 'loc_s2'], ['subj_s1', 'clo_s2'],
             ['verb_s1', 'subj_s2'], ['verb_s1', 'verb_s2'], ['verb_s1', 'obj_s2'], ['verb_s1', 'loc_s2'], ['verb_s1', 'clo_s2'],
             ['obj_s1', 'subj_s2'], ['obj_s1', 'verb_s2'], ['obj_s1', 'obj_s2'], ['obj_s1', 'loc_s2'], ['obj_s1', 'clo_s2'],
             ['loc_s1', 'subj_s2'], ['loc_s1', 'verb_s2'], ['loc_s1', 'obj_s2'], ['loc_s1', 'loc_s2'], ['loc_s1', 'clo_s2'],
             ['clo_s1', 'subj_s2'], ['clo_s1', 'verb_s2'], ['clo_s1', 'obj_s2'], ['clo_s1', 'loc_s2'], ['clo_s1', 'clo_s2']]
    pairs_map = {'subj_s1': 'the subject of sentence 1',
                 'verb_s1': 'the verb of sentence 1',
                 'obj_s1': 'the object of sentence 1',
                 'loc_s1': 'the location of sentence 1',
                 'clo_s1': 'the clothing described in sentence 1',
                 'subj_s2': 'the subject of sentence 2',
                 'verb_s2': 'the verb of sentence 2',
                 'obj_s2': 'the object of sentence 2',
                 'loc_s2': 'the location of sentence 2',
                 'clo_s2': 'the clothing described in sentence 2',}
    Sentence2_indices_map = ['subj_s2', 'verb_s2', 'obj_s2', 'loc_s2', 'clo_s2']
    reasons = list()
    if y_hat_pred == 'neutral':
        for i, subphrase_indices in enumerate(Sentence2_indices):
            if all([Z[k] == 'neutral' or Z[k] == 'nan' or pd.isnull(Z[k]) for k in subphrase_indices]) and any([Z[k] == 'neutral' for k in subphrase_indices]):
                reasons += [f'there is no indication that {pairs_map[Sentence2_indices_map[i]]} is {subphrases["string_" + Sentence2_indices_map[i]]}']
    elif y_hat_pred == 'contradiction':
        for i,z in enumerate(Z):
            if z == 'contradiction':
                if i in non_mixed_pairs_indices:
                    reasons += [f'{subphrases["string_" + pairs[use_z_values[i]][0]].lower()} is not the same as {subphrases["string_" + pairs[use_z_values[i]][1]].lower()}', ]
                else:
                    reasons += [f'if {pairs_map[pairs[use_z_values[i]][0]]} is {subphrases["string_" + pairs[use_z_values[i]][0]].lower()}, {pairs_map[pairs[use_z_values[i]][1]]} cannot be {subphrases["string_" + pairs[use_z_values[i]][1]].lower()}', ]
    elif y_hat_pred == 'entailment':
            for i,z in enumerate(Z):
                if z == 'entailment':
                    if i in non_mixed_pairs_indices:
                        reasons += [f'{subphrases["string_" + pairs[use_z_values[i]][0]].lower()} is the same as {subphrases["string_" + pairs[use_z_values[i]][1]].lower()}', ]
                    else:
                        reasons += [f'if {pairs_map[pairs[use_z_values[i]][0]]} is {subphrases["string_" + pairs[use_z_values[i]][0]].lower()}, then {pairs_map[pairs[use_z_values[i]][1]]} has to be {subphrases["string_" + pairs[use_z_values[i]][1]].lower()}', ]
    return " and ".join(reasons)

In [51]:
np.mean(y_hat_pred == y_hat)

0.5512355848434926

In [52]:
f1_score(y_hat, y_hat_pred, average='macro')

0.5489621098111873

In [35]:
dev.loc[dev_subphrases.index[2]]

gold_label                                                     contradiction
Sentence1                  Two women are embracing while holding to go pa...
Sentence2                               The men are fighting outside a deli.
Explanation_1              In the first sentence there is an action of af...
Sentence1_marked_1         Two *women* are *embracing* while holding to g...
Sentence2_marked_1                  The *men* are *fighting* outside a deli.
Sentence1_Highlighted_1                                                  1,3
Sentence2_Highlighted_1                                                  1,3
Explanation_2              Women are different than men and embracing is ...
Sentence1_marked_2         *Two* *women* are *embracing* while holding to...
Sentence2_marked_2                  The *men* are *fighting* outside a deli.
Sentence1_Highlighted_2                                                0,1,3
Sentence2_Highlighted_2                                                  1,3

In [59]:
dev_subphrases.loc['4345676156.jpg#4r1e']

Unnamed: 0                        1235
string_subj_s1               Three men
string_verb_s1                gathered
string_obj_s1                      NaN
string_loc_s1                      NaN
string_clo_s1                      NaN
string_subj_s2    a piece of furniture
string_verb_s2                  gather
string_obj_s2        A bunch of people
string_loc_s2                      NaN
string_clo_s2                      NaN
Name: 4345676156.jpg#4r1e, dtype: object

In [58]:
for i in range(404,410):
    if y_hat[i] != y_hat_pred[i]:
        suffix = 'WRONG: '
    else:
        suffix = 'TRUE: '
    print(suffix + generate_explanation(z[i,:], dev_subphrases.iloc[i]))

TRUE: there is no indication that the verb of sentence 2 is smuggle
TRUE: there is no indication that the verb of sentence 2 is carrying
WRONG: there is no indication that the verb of sentence 2 is doing and there is no indication that the object of sentence 2 is marching
WRONG: there is no indication that the verb of sentence 2 is gather
WRONG: there is no indication that the verb of sentence 2 is gather
TRUE: there is no indication that the subject of sentence 2 is their weekly meeting and there is no indication that the verb of sentence 2 is gather


In [23]:
human_z = pd.read_csv("manual_z_values.csv", sep=";").to_numpy()
for i in range(human_z.shape[0]):
    for j in range(human_z.shape[1]):
        if human_z[i,j] == "n":
            human_z[i,j] = "neutral"
        elif human_z[i,j] == "e":
            human_z[i,j] = "entailment"
        elif human_z[i,j] == "c":
            human_z[i,j] = "contradiction"

y_hat_human_pred = predict_y_from_z(human_z)

In [54]:
human_z

array([['neutral', 'neutral', 'neutral', nan, nan, 'neutral',
        'entailment', 'neutral', nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan],
       ['entailment', 'neutral', 'neutral', nan, nan, 'neutral',
        'neutral', 'neutral', nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan],
       ['contradiction', 'neutral', 'neutral', nan, nan, 'neutral',
        'contradiction', nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan],
       ['entailment', 'neutral', 'neutral', nan, nan, 'neutral',
        'entailment', 'neutral', nan, nan, 'neutral', 'neutral',
        'entailment', nan, nan, 'neutral', 'neutral', 'neutral', nan,
        nan, 'entailment', 'neutral', 'neutral', nan, nan],
       ['entailment', 'neutral', 'neutral', nan, nan, 'neutral',
        'neutral', 'neutral', nan, nan, 'neutral', 'neutral', 'neutral',
        nan, nan, 'neutral', 'ne

In [64]:
for i in range(40):
    if y_hat[i] != y_hat_human_pred[i]:
        suffix = 'WRONG: '
    else:
        suffix = 'TRUE: '
    print(suffix + generate_explanation(human_z[i,:], dev_subphrases.iloc[i]))

TRUE: there is no indication that the subject of sentence 2 is The sisters and there is no indication that the object of sentence 2 is goodbye
WRONG: there is no indication that the verb of sentence 2 is holding and there is no indication that the object of sentence 2 is packages
TRUE: two women is not the same as the men and embracing is not the same as fighting
TRUE: a woman is the same as a woman and doing is the same as doing and a cartwheel is the same as a cartwheel and if the clothing described in sentence 1 is bikini, then the subject of sentence 2 has to be a woman
WRONG: there is no indication that the verb of sentence 2 is fixing and there is no indication that the object of sentence 2 is her home
TRUE: if the object of sentence 1 is a cartwheel, the location of sentence 2 cannot be on her head and in the sand is not the same as on her head
TRUE: surgery on patient is not the same as lunch
TRUE: two doctors is the same as doctors and perform is the same as performing and sur

IndexError: index 30 is out of bounds for axis 0 with size 30

In [24]:
y_hat_human_pred[np.where(y_hat[:30] != y_hat_human_pred)]

array(['neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral'],
      dtype=object)

In [25]:
y_hat.iloc[np.where(y_hat[:30] != y_hat_human_pred)]

pairID
4705552913.jpg#2r1e       entailment
3948003394.jpg#1r1c    contradiction
7391785714.jpg#4r1e       entailment
5777129645.jpg#2r1n       entailment
5777129645.jpg#2r1e       entailment
5777129645.jpg#2r1c    contradiction
Name: gold_label, dtype: object

In [26]:
human_z[y_hat[:30] != y_hat_human_pred,:]

array([['entailment', 'neutral', 'neutral', nan, nan, 'neutral',
        'neutral', 'neutral', nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan],
       ['entailment', 'neutral', 'neutral', nan, nan, 'neutral',
        'neutral', 'neutral', nan, nan, 'neutral', 'neutral', 'neutral',
        nan, nan, 'neutral', 'neutral', 'neutral', nan, nan,
        'entailment', 'neutral', 'neutral', nan, nan],
       ['entailment', 'neutral', 'neutral', nan, nan, 'neutral',
        'neutral', 'neutral', nan, nan, nan, nan, nan, nan, nan,
        'neutral', 'neutral', 'neutral', nan, nan, nan, nan, nan, nan,
        nan],
       ['neutral', 'neutral', nan, 'neutral', nan, 'neutral',
        'entailment', nan, 'neutral', nan, nan, nan, nan, nan, nan,
        'neutral', 'neutral', nan, 'entailment', nan, nan, nan, nan, nan,
        nan],
       ['entailment', 'neutral', 'neutral', nan, nan, 'neutral',
        'entailment', 'neutral', nan, nan, nan, nan, nan, n

In [27]:
dev_subphrases.iloc[np.where(y_hat[:30] != y_hat_human_pred)]

,Unnamed: 0,string_subj_s1,string_verb_s1,string_obj_s1,string_loc_s1,string_clo_s1,string_subj_s2,string_verb_s2,string_obj_s2,string_loc_s2,string_clo_s2
pairID,,,,,,,,,,,
4705552913.jpg#2r1e,1,Two women,embracing,NaN,NaN,NaN,Two woman,holding,packages,NaN,NaN
3948003394.jpg#1r1c,19,A woman,doing,a cartwheel,in the sand,bikini,A woman,fixing,her home,NaN,NaN
7391785714.jpg#4r1e,50,Kids,are,NaN,on a amusement ride,NaN,Kids,ride,an amusement ride,NaN,NaN
5777129645.jpg#2r1n,86,The two farmers,working,NaN,on a piece of John Deere equipment,NaN,Men,working,NaN,on John Deere equipment,NaN
5777129645.jpg#2r1e,87,The two farmers,working,NaN,on a piece of John Deere equipment,NaN,two farmers,work,Deere equipment,NaN,NaN
5777129645.jpg#2r1c,88,The two farmers,working,NaN,on a piece of John Deere equipment,NaN,2 Men,making,a pool,NaN,NaN


In [30]:
train

,pairID,gold_label,Sentence1,Sentence2,Explanation_1,WorkerId,Sentence1_marked_1,Sentence2_marked_1,Sentence1_Highlighted_1,Sentence2_Highlighted_1
0,3416050480.jpg#4r1n,neutral,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,the person is not necessarily training his horse,AF0PI3RISB5Q7,A person on a horse jumps over a broken down a...,A person is *training* *his* *horse* for a co...,{},"3,4,5"
1,3416050480.jpg#4r1c,contradiction,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",One cannot be on a jumping horse cannot be a d...,A36ZT2WFIA2HMF,A person *on* *a* *horse* *jumps* over a brok...,"A person *is* *at* *a* *diner,* *ordering* an...","4,2,3,5","2,5,4,3,6"
2,3416050480.jpg#4r1e,entailment,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",a broken down airplane is outdoors,A2GK75ZQTX2RDZ,A person on a horse jumps over *a* *broken* *...,"A person is *outdoors,* on a horse.","8,9,10,7",3
3,2267923837.jpg#2r1n,neutral,Children smiling and waving at camera,They are smiling at their parents,Just because they are smiling and waving at a ...,A18TOIDG32QICP,Children smiling and waving at camera,They are smiling *at* *their* *parents*,{},"5,3,4"
4,2267923837.jpg#2r1e,entailment,Children smiling and waving at camera,There are children present,The children must be present to see them smili...,AEX0YE6TUZRHT,*Children* *smiling* *and* *waving* at camera,There are children *present*,"0,1,3,2",3
...,...,...,...,...,...,...,...,...,...,...
289363,2267923837.jpg#3r1c,contradiction,Four dirty and barefooted children.,four kids won awards for 'cleanest feet',in a both sentence dirty and cleanest are not ...,A8Q7V9A0SN3SF,Four *dirty* and barefooted children.,four kids won awards for *'cleanest* feet',1,5
289364,2267923837.jpg#3r1n,neutral,Four dirty and barefooted children.,"four homeless children had their shoes stolen,...",the children are not necessarily homeless,AF0PI3RISB5Q7,Four dirty and barefooted children.,four *homeless* children had their shoes stol...,{},1
289365,7979219683.jpg#2r1n,neutral,A man is surfing in a bodysuit in beautiful bl...,A man in a bodysuit is competing in a surfing ...,the man is not necessarily competing,AF0PI3RISB5Q7,A man is surfing in a bodysuit in beautiful bl...,A man in a bodysuit is *competing* in a surfi...,{},6
289366,7979219683.jpg#2r1c,contradiction,A man is surfing in a bodysuit in beautiful bl...,A man in a business suit is heading to a board...,That is either a business suit or bodysuit.,A2BT0LQYMBL7LV,A man is surfing in a *bodysuit* in beautiful...,A man in a *business* *suit* is heading to a ...,6,"4,5"


In [28]:
np.mean(y_hat_pred[:30] == y_hat_human_pred)

NameError: name 'y_hat_pred' is not defined

In [29]:
np.mean(y_hat[:30] == y_hat_human_pred)

0.8

In [86]:
np.mean(y_hat_pred == y_hat)

0.4688632619439868

In [98]:
z

array([['entailment', 'entailment', 'entailment', ..., 'nan', 'nan',
        'nan'],
       ['entailment', 'entailment', 'entailment', ..., 'nan', 'nan',
        'nan'],
       ['entailment', 'entailment', 'nan', ..., 'nan', 'nan', 'nan'],
       ...,
       ['entailment', 'entailment', 'nan', ..., 'nan', 'nan', 'nan'],
       ['entailment', 'entailment', 'entailment', ..., 'nan', 'nan',
        'nan'],
       ['entailment', 'entailment', 'entailment', ..., 'nan', 'nan',
        'nan']], dtype='<U100')

In [99]:
human_z

array([['neutral', 'neutral', 'neutral', nan, nan, 'neutral',
        'entailment', 'neutral', nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan],
       ['entailment', 'neutral', 'neutral', nan, nan, 'neutral',
        'neutral', 'neutral', nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan],
       ['contradiction', 'neutral', 'neutral', nan, nan, 'neutral',
        'contradiction', nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan],
       ['entailment', 'neutral', 'neutral', nan, nan, 'neutral',
        'entailment', 'neutral', nan, nan, 'neutral', 'neutral',
        'entailment', nan, nan, 'neutral', 'neutral', 'neutral', nan,
        nan, 'entailment', 'neutral', 'neutral', nan, nan],
       ['entailment', 'neutral', 'neutral', nan, nan, 'neutral',
        'neutral', 'neutral', nan, nan, 'neutral', 'neutral', 'neutral',
        nan, nan, 'neutral', 'ne

In [104]:
dev_subphrases.iloc[39]

Unnamed: 0                        43
string_subj_s1           Two doctors
string_verb_s1               perform
string_obj_s1     surgery on patient
string_loc_s1             on patient
string_clo_s1                    NaN
string_subj_s2               Doctors
string_verb_s2            performing
string_obj_s2                surgery
string_loc_s2                    NaN
string_clo_s2                    NaN
Name: 5436250638.jpg#4r1e, dtype: object